In [16]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from src.data_utils import load_data

In [2]:
class CustomArcticDataset(torch.utils.data.Dataset):
    def __init__(self, data, sequence_length=30, forecast_horizon=1, features=None, scaler = None):
        self.data = data.sort_values('date').reset_index(drop=True)
        self.sequence_length = sequence_length
        self.forecast_horizon = forecast_horizon

        if features is None:
            self.features = ['extent_mkm2']
        else:
            self.features = features

        self.data = self.data[self.features].values.astype(np.float32)
        if scaler is None:
            self.mean = self.data.mean(axis=0)
            self.std = self.data.std(axis=0)
        else:
            self.mean, self.std = scaler

        self.data = (self.data - self.mean) / self.std

    def __len__(self):
        return len(self.data) - self.sequence_length - self.forecast_horizon + 1

    def __getitem__(self, idx):
        X = self.data[idx:idx + self.sequence_length]
        y = self.data[idx + self.sequence_length + self.forecast_horizon - 1][0]

        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        return X, y


In [3]:
train_data = load_data(regions='pan_arctic', years=range(1989, 2020))
test_data = load_data(regions='pan_arctic', years=range(2020, 2024))

In [4]:
train_dataset = CustomArcticDataset(train_data, sequence_length=30, forecast_horizon=1)
test_dataset = CustomArcticDataset(test_data, scaler=(train_dataset.mean, train_dataset.std))

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
for X_batch, y_batch in train_loader:
    print(f"Batch X shape: {X_batch.shape}")  # (32, 30, 1)
    print(f"Batch y shape: {y_batch.shape}")  # (32, 1)
    break


Batch X shape: torch.Size([32, 30, 1])
Batch y shape: torch.Size([32])


In [12]:
class IceExtentLSTM(torch.nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2, output_size=1):
        super(IceExtentLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers,
                                  batch_first=True, dropout=0.2)
        self.fc = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])
        return out

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model = IceExtentLSTM(input_size=1, hidden_size=64, num_layers=2, output_size=1)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")


Using device: cpu
Model parameters: 50497


In [14]:
num_epochs = 100
best_val_loss = float('inf')
patience = 15
patience_counter = 0

train_losses = []
val_losses = []

print("\nStarting training...")
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        predictions = model(X_batch)
        loss = criterion(predictions.squeeze(), y_batch)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            predictions = model(X_batch)
            loss = criterion(predictions.squeeze(), y_batch)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(test_loader)

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

    scheduler.step(avg_val_loss)

    if (epoch + 1) % 5 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'  Train Loss: {avg_train_loss:.6f}')
        print(f'  Val Loss: {avg_val_loss:.6f}')

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_model.pt')
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

print(f"\nTraining complete! Best validation loss: {best_val_loss:.6f}")


Starting training...
Epoch 5/100
  Train Loss: 0.002138
  Val Loss: 0.001069
Epoch 10/100
  Train Loss: 0.001377
  Val Loss: 0.000824
Epoch 15/100
  Train Loss: 0.001160
  Val Loss: 0.000643
Epoch 20/100
  Train Loss: 0.000950
  Val Loss: 0.000654
Epoch 25/100
  Train Loss: 0.000897
  Val Loss: 0.000946
Epoch 30/100
  Train Loss: 0.000626
  Val Loss: 0.000424
Epoch 35/100
  Train Loss: 0.000624
  Val Loss: 0.000373
Epoch 40/100
  Train Loss: 0.000613
  Val Loss: 0.000864
Epoch 45/100
  Train Loss: 0.000522
  Val Loss: 0.000473
Epoch 50/100
  Train Loss: 0.000519
  Val Loss: 0.000385
Epoch 55/100
  Train Loss: 0.000513
  Val Loss: 0.000328
Epoch 60/100
  Train Loss: 0.000491
  Val Loss: 0.000347
Epoch 65/100
  Train Loss: 0.000462
  Val Loss: 0.000410
Epoch 70/100
  Train Loss: 0.000429
  Val Loss: 0.000338
Epoch 75/100
  Train Loss: 0.000429
  Val Loss: 0.000328
Epoch 80/100
  Train Loss: 0.000420
  Val Loss: 0.000321
Epoch 85/100
  Train Loss: 0.000413
  Val Loss: 0.000318
Epoch 90/1